In [183]:

import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import time

nltk.download('averaged_perceptron_tagger')

#캐릭터 수 카운트하기
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimkwangil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [184]:

def character_all_section(text):

    # Number of Characters
    def NumberofCharacters(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        ##print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        ##print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        #print (filtered_chr_text__) # 중복값 제거 확인
        
        # for i in filtered_chr_text__:
        #     ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
        
        print("중복제거된 캐릭터 표현 수:", char_count_)
        
        #전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율 계산, 그 비율에 맞게 결과값 조정
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        print("전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율:", result_char_ratio)
        # 전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율: 12.36
        
        ###################################
        ########### 추가한 코드 ##############
        ###################################
        #이름을 인식하여 캐릭터 수 카운트하기
        doc = nlp(text)
        ext_label_char =[(X.text, X.label_) for X in doc.ents] #레이블 추출
        ext_label_char = set(ext_label_char) #같은 이름 중복 제거
        print("레이블 추출:", ext_label_char)
        
        cont = 0 #캐릭터 수 초기화
        for i in ext_label_char:
            'PERSON' in i # PERSON이 튜플 안에 있다면 캐릭터니까 카운트를 한다. 
            cont += 1
            
        print("에세이에 포함된 이름 수:", cont)
        #총 캐릭터수 계산(character_list + cont) 합치면
        char_count_ = char_count_ + cont
        char_count_ = round(char_count_ / total_words * 100, 2) #전체문장길이 캐릭터 비율 적용
        
        print("1명 에세이 총 캐릭터수:", char_count_) 
        return char_count_



    # number_of_characters = NumberofCharacters(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # number_of_characters
    # #print ('=============================================')
    # #print ('Number of Characters :', number_of_characters)
    # #print ('=============================================')

    ####################################
    #### Character Descriptiveness #####
    ####################################

    def character_descrip(text):

        input_sentence = text

        def findSentence(input_sentence):
            result = []

            data = str(input_sentence)
            #data = input_sentence.splitlines()
            
            findText = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']


            sentences = data.split(".")
            
            for sentence in sentences:
                for item in findText:
                    if item in sentence:
                        result.append(sentence)

            return result

        input_sent_included_character = findSentence(text) 
        input_sent_chr = set(input_sent_included_character) #중복값을 제거해보자
        input_sent_chr = '.'.join(input_sent_chr) #하나의 문자열로 합쳐야 원본 문장처럼 변환되고, 이것을 show/tell 분석코드에 넣게됨



        #입력된 전체 문장을 개별문장으로 분리하여 전처리 처리함
        def sentence_to_df(input_sentence):

            input_text_df = nltk.tokenize.sent_tokenize(input_sentence)
            test = []

            for i in range(0,len(input_text_df)):
                new_label = np.random.randint(0,2)  # 개별문장(input_text_df) 수만큼 0 또는 1 난수 생성
                data = [new_label, input_text_df[i]]
                test.append(data)

            ##print(test)
            dataf = pd.DataFrame(test, columns=['label', 'text'])
            ##print(dataf)
            return dataf


        class STDataset(Dataset):
            ''' Showing Telling Corpus Dataset '''
            def __init__(self, df):
                self.df = df

            def __len__(self):
                return len(self.df)

            def __getitem__(self, idx):
                text = self.df.iloc[idx, 1]
                label = int(self.df.iloc[idx, 0])
                return text, label


        ###########입력받은 데이터 처리 실행하는 메소드 showtell_classfy() ###############
        #result_all.html에서 입력받을 text를 contents에 넣고 전처리 후 데이터프레임에 넣어줌
        def showtell_classfy(text):
            contents = str(text)
            preprossed_contents_df = sentence_to_df(contents)

            preprossed_contents_df.dropna(inplace=True)
            #전처리된 데이터를 확인(데이터프레임으로 가공됨)
            preprossed_contents_df__ = preprossed_contents_df.sample(frac=1, random_state=999)
            

            #파이토치에 입력하기 위해서 로딩...
            ST_test_dataset = STDataset(preprossed_contents_df__)
            test_loader = DataLoader(ST_test_dataset, batch_size=1, shuffle=True, num_workers=0)
            #로딩되는지 확인
            ST_test_dataset.__getitem__(1)

            #time.sleep(1)



            #check whether cuda is available
            #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
            device = torch.device("cpu")  
            #device = torch.device("cuda")
            #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
            tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
            model = BertForSequenceClassification.from_pretrained('bert-large-cased')
            model.to(device)



            # for text, label in test_loader :
            #     #print("text:",text)
            #     #print("label:",label)


            #저장된 모델을 불러온다.
            #J:\Django\EssayFit_Django\essayfitaiproject\essayfitapp\model.pt
            #time.sleep(1)
            #model = torch.load("model.pt", map_location=torch.device('cpu'))
            model = torch.load("/Users/kimkwangil/Documents/001_ESSAYFITAI/EssayFitAI_2021-01-08/ai_server_backup/essayfit/essayai/data/model.pt", map_location=torch.device('cpu'))
            # #print("model loadling~")
            model.eval()


            pred_loader = test_loader
            # #print("pred_loader:", pred_loader)
            total_loss = 0
            total_len = 0
            total_showing__ = 0
            total_telling__ = 0

            showing_conunter = [] #문장에 해당하는 SHOWING을 계산한다.
            
            # #print("check!")
            for text, label in pred_loader:
                # #print("text:",text)
                ##print("label:",label)
                encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] #text to tokenize
                padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] #padding
                sample = torch.tensor(padded_list) #torch tensor로 변환
                sample, label = sample.to(device), label.to(device) #tokenized text에 label을 넣어서 Device(gpu/cpu)에 넣기 위해 준비
                labels = torch.tensor(label) #레이블을 텐서로 변환
                #time.sleep(1)
                outputs = model(sample,labels=labels) #모델을 통해서 샘플텍스트와 레이블 입력데이터를 출력 output에 넣음
                #시간 딜레이를 주자
                #time.sleep(1)
                _, logits = outputs #outputs를 로짓에 넣음 이것을 softmax에 넣으면 0~1 사이로 결과가 출력됨
                
                pred = torch.argmax(F.softmax(logits), dim=1) #드디어 예측한다. argmax는 리스트(계산된 값)에서 가장 큰 값을 추출하여 pred에 넣는다. 0 ~1 사이의 값이 나올거임
                # #print('pred :', pred)
                # correct = pred.eq(labels) 
                showing__ = pred.eq(1) # 예측한 결과가 1과 같으면 showing이다   >> TRUE   SHOWING을 추출하려면 이것만 카운드하면 된다. 
                telling__ = pred.eq(0) # 예측한 결과가 0과 같으면 telling이다   >> FALSE
                
                ##print('showing : ', showing__)
                ##print('telling : ', telling__)
                
                
                showing_conunter.append(text)        
                #pred_ = round(float(pred))
                showing_conunter.append(pred)


            return showing_conunter 


        st_re = showtell_classfy(str(input_sent_chr)) # 캐릭터거 포함된 문장(전처리 완료된) 입력

        df = DataFrame(st_re)
        df_ = df[0::2] # 글만 추출
        df_label = df[1::2] # 레이블만 추출

        df_.reset_index(drop=True, inplace=True) #데이터를 합치기 위해서 초기화
        df_label.reset_index(drop=True, inplace=True)

        df_result = pd.concat([df_,df_label],axis=1) #합치기

        df_result.columns = ['sentence','show/tell']

        df_fin = df_result['show/tell'].value_counts(normalize=True)
        list(df_fin)
        showing_sentence_with_char = max(round(df_fin*100))

        # #print("===============================================================")
        # #print ('Character Descriptiveness : ', showing_sentence_with_char)
        # #print("===============================================================")

        return showing_sentence_with_char



    ################################################
    #############  Emphasis on YOU  ################
    ################################################
    def EmphasisOnYou(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'I', 'my', 'me', 'mine', 'one']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        ##print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        ##print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        #filtered_chr_text_ = set(filtered_chr_text) #중복제거
        #filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        # #print (filtered_chr_text__) # 중복값 제거 확인
        
        # for i in filtered_chr_text__:
        #     ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수

        # char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수

        result_char_ratio = round(char_total_count/total_words * 100, 2)
        print("전체 문장에서 캐릭터 단어 비율::::::", result_char_ratio)

        print("emphasis on you", char_total_count)
        
        
        char_total_count = round(char_total_count / total_words * 100, 2) #전체문장길이 캐릭터 비율 적용
            
            
        return char_total_count


    # EmphasisOnYou_ = EmphasisOnYou(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnYou_
    # #print ('=============================================')
    # #print ('Emphasis on You :', EmphasisOnYou_)
    # #print ('=============================================')



    #########################################
    ######### Emphasis on others  ###########
    #########################################
    def EmphasisOnOthers(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['they','them','he','him','his' 'she','her','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin',
                        'daughter', 'duplicate', 'ex', 'father', 'forefather', 'founder', 'gemini',
                        'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium', 'husband',
                        'match', 'mother', 'nephew', 'niece', 'parent', 'person', 'rear',
                        'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow',
                        'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        ##print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        ##print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        # filtered_chr_text_ = set(filtered_chr_text) #중복제거
        # filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        #print (filtered_chr_text__) # 중복값 제거 확인
        
        # for i in filtered_chr_text__:
        #     ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        print("Emphasis on others 수집 단어들:",filtered_chr_text)
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        print("Emphasis on others:", char_total_count)
    
        char_total_count = round(char_total_count / total_words * 100, 2) #전체문장길이 캐릭터 비율 적용
    
        return char_total_count


    # EmphasisOnOthers_ = EmphasisOnOthers(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnOthers_
    # #print ('=============================================')
    # #print ('Emphasis on Others :', EmphasisOnOthers_)
    # #print ('=============================================')


    character_descriptiveness = character_descrip(text)
    # #print("===============================================================")
    # #print ('Character Descriptiveness : ' , character_descriptiveness)
    # #print("===============================================================")


    number_of_characters = NumberofCharacters(text) 
    # #print ('=============================================')
    # #print ('Number of Characters :' , number_of_characters)
    # #print ('=============================================')


    EmphasisOnYou_ = EmphasisOnYou(text)
    # #print ('=============================================')
    # #print ('Emphasis on You :' , EmphasisOnYou_)
    # #print ('=============================================')


    EmphasisOnOthers_ = EmphasisOnOthers(text) 
    # #print ('=============================================')
    # #print ('Emphasis on Others :' , EmphasisOnOthers_)
    # #print ('=============================================')


    return character_descriptiveness, number_of_characters, EmphasisOnYou_, EmphasisOnOthers_


In [185]:
# 1) 학생 #1
# -성적/활동 높음
# -캐릭터 수 엄청 많음 (15명) >>>>>>>>> result_number_of_chracters': 0 >>>>> 2로 나와야 정상
# -Setting 갯수 엄청 많음 / Setting 설명 엄청 많음
# -Conflict 엄청 많음 (Body에 집중)
# -Emotion (negative 한게 엄청 많음... 약간의 positive)
# -Essay Prompt #2 (Obstacle) - Theme 맞는지 (각 prompt 별로 emotion / keyword 조건 확인)


text= """Jennifer walked into the dark and creepy bedroom, filled with old furniture and dust. She felt uncomfortable because she knew that this old house belonged to John. John was the cousin of Tom, who used to abuse her very cruelly.The wooden house was built with cherry wood. It was previously owned by criminals, named Joe and Cyndi. They were grandparents of John. The basement was filled with screams and sobbing, as the angry captives, such as Roy, Eugene, and Peter, who were miserably tortured in the wet, cold floor of the isolated cell. They felt so threatened as the tension heightened. The pressure was high as they violently hit each other and bled in the faces. The cement floor of the bathroom was filled with a red pungent smell of blood and vivid marks of ripped flesh splashed everywhere. They ran like turkey without heads but began to brutally crash into the steel walls and gates, collapsing into the pit. There were fierce yelling and fighting but Bill and Ted didn't seem to care.
Finally, the room quieted down as people calmed down too. It was quiet and peaceful. Everyone seemed to be sleeping comfortably, with a smile on their faces. I am happy now because I am no longer angry."""


# 중복제거된 캐릭터 표현 수: 7
# 전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율: 6.69
# 레이블 추출: {('Bill', 'PERSON'), ('John', 'PERSON'), ('Tom', 'PERSON'), ('Eugene', 'WORK_OF_ART'), ('Peter', 'PERSON'), ('Jennifer', 'PERSON'), ('turkey', 'GPE'), ('Ted', 'PERSON'), ('Roy', 'PERSON'), ('Joe', 'PERSON'), ('Cyndi', 'GPE')}
# 에세이에 포함된 이름 수: 11
# 1명 에세이 총 캐릭터수: 7.53
# 전체 문장에서 캐릭터 단어 비율:::::: 0.84
# emphasis on you 2
# Emphasis on others 수집 단어들: ['cousin', 'cousin', 'her', 'they', 'they', 'they', 'they', 'their']
# Emphasis on others: 8
# 한개의 에세이 캐릭터 수: 7.53
# 1000명 평균 캐릭터수_입력문장길이비율적용: 7
# 캐릭터 분석 결과 (1, 5)
# 캐랙터 상대적 점수 1
# 최종결과 :  {'number_of_chracters': 7.53, 'character_description': 36.0, 'emaphasis_on_you': 0.84, 
#          'emaphasis_on_others': 3.35, 'result_number_of_chracters': 1, 'result_character_description': 1,
#          'result_emaphasis_on_you': 0, 'result_emaphasis_on_others': 0, 'avg_character': 2.5}


In [186]:
# 2) 학생 #2
# -성적/활동 낮음
# -캐릭더 수 없음 (나만, I)   -------> 'result_number_of_chracters': 0 으로 나와야 함
# -Setting 갯수 별로 없음 / Setting 설명 없음
# -Conflict 거의 없음 (neutral)
# -Emotion (거의 realization 쪽으로... intellectual/academic 하게)
# -Essay Prompt #3 (Intellectual) - Theme 맞는지  (각 prompt 별로 emotion / keyword 조건 확인)

# text = """I always studied physics because it's science. When objects meet each other, there starts some type of force. And forces make things to move, hence resulting in perpetual movements using gravity.
# Gravity is a unique concept because I think it applies to everything around us. Objects are definitely affected by gravity and I understood this from a long time ago. This is why physics deals with gravity and objects. We can now see that physics is a study of relationships, more physical than philosophical, because it is proven to us through research. Therefore, the lesson in physics is something I discovered and I realize that it is important for engineering as well.
# In terms of engineering, physics is used as the foundation. Furthermore, the foundation for physics is math. Therefore, math is very relevant with engineering. This is a lesson I found to be true and I will remember this forever.
# """

# 중복제거된 캐릭터 표현 수: 3
# 전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율: 12.21
# 레이블 추출: set()
# 에세이에 포함된 이름 수: 0
# 1명 에세이 총 캐릭터수: 1.74
# 전체 문장에서 캐릭터 단어 비율:::::: 4.07
# emphasis on you 7
# Emphasis on others 수집 단어들: []
# Emphasis on others: 0
# 한개의 에세이 캐릭터 수: 1.74
# 1000명 평균 캐릭터수_입력문장길이비율적용: 7
# 캐릭터 분석 결과 (0, 1)
# 캐랙터 상대적 점수 0
# 최종결과 :  {'number_of_chracters': 1.74, 'character_description': 45.0, 'emaphasis_on_you': 4.07, 
#          'emaphasis_on_others': 0.0, 'result_number_of_chracters': 0, 'result_character_description': 0, 
#          'result_emaphasis_on_you': 1, 'result_emaphasis_on_others': 0, 'avg_character': 2.5}
    

In [187]:
# 학생 #3 에세이

# -성적/활동 중간
# -캐릭터 수 4명 (중간)   >>>>> 결과: result_number_of_chracters': 1  
# -Setting 갯수 설명 중간 (3개 장소)
# -Conflict 중간
# -Emotion (positive & realization 쪽으로)
# -Essay Prompt #1 (Identity) - Theme 맞는지  (각 prompt 별로 emotion / keyword 조건 확인)

# text = """Meeting a friend is always a joy in my life. Yesterday, I left to see Thomas and Sammy. They are my best friends whom I treasure dearly. The paved road was so straight that it was easy and pleasant for me to walk on.
# When I saw them at the bright red restaurant, I knew the waiter would serve us ketchup. I love ketchup because it goes perfectly with french fries, which was made in France. Thomas and Sammy brought another friend named Kim. All of us spent a wonderful time, sitting in the hall and listening to music. The large hall at the building has very beautiful windows and we loved it.
# As we came back to the steep hill where the blazing sun was setting, we were grateful for the lesson we found. The hill was connected to a mountain which told us that friendship is the pathway to the high grounds. It was a beautiful day with friends."""


# 중복제거된 캐릭터 표현 수: 7
# 전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율: 12.36
# 레이블 추출: {('Yesterday', 'DATE'), ('Thomas', 'PERSON'), ('french', 'NORP'), ('Sammy', 'PERSON'), ('a beautiful day', 'DATE'), ('Kim', 'PERSON'), ('France', 'GPE')}
# 에세이에 포함된 이름 수: 7
# 1명 에세이 총 캐릭터수: 7.87
# 전체 문장에서 캐릭터 단어 비율:::::: 4.49
# emphasis on you 8
# Emphasis on others 수집 단어들: ['they', 'them']
# Emphasis on others: 2
# 한개의 에세이 캐릭터 수: 7.87
# 1000명 평균 캐릭터수_입력문장길이비율적용: 7
# 캐릭터 분석 결과 (1, 5)
# 캐랙터 상대적 점수 1
# 최종결과 :  {'number_of_chracters': 7.87, 'character_description': 64.0, 'emaphasis_on_you': 4.49,
#          'emaphasis_on_others': 1.12, 'result_number_of_chracters': 1, 'result_character_description': 1, 
#          'result_emaphasis_on_you': 1, 'result_emaphasis_on_others': 0, 'avg_character': 4.0}
    
    

In [188]:
def character_total_analysis(text):

    ##### 주석 처리 ##########################
    ########################################
    char_sec_re = character_all_section(text)

    # #print("1명의 에세이 결과 계산점수 :", char_sec_re)

    # 위에서 계산한 총 4개의 값을 개인, 그룹의 값과 비교하여 lacking, ideal, overboard 계산
    
    # 개인에세이 값 계산 4가지 결과 추출 >>>>> personal_value 로 입력됨
    one_ps_char_desc = char_sec_re[0] # 튜플에서 첫번재 인댁스 값 가져오기 : Character Descriptiveness
    one_ps_num_of_char = char_sec_re[1]
    print('한개의 에세이 캐릭터 수:',one_ps_num_of_char)
    one_ps_emp_on_you = char_sec_re[2]
    one_ps_emp_on_others = char_sec_re[3]

    ##############################################################################################################################
    ## 1000명 데이터의 각 값(char_desc_mean)의 평균 값 전달. >>>> group_mean 으로 입력됨
    char_desc_mean = [77, 7, 3, 10] # 현재 계산 완료한 1000명의 평균 값(고정값)  1명 에세이 총 캐릭터수: 173.04
    group_db_fin_result = [5.0] #레이다차트의 1000명 평균값 기준설정
    ##############################################################################################################################


    char_desc_ideal_mean = char_desc_mean[0] #첫번째 값을 가져옴, Character Descriptiveness
    num_of_char_ideal_mean = char_desc_mean[1] #Number of Characters
    print('1000명 평균 캐릭터수_입력문장길이비율적용:', num_of_char_ideal_mean)
    emp_on_you_ideal_mean = char_desc_mean[2] #Emphasis on You 
    emp_on_others_ideal_mean = char_desc_mean[3] #Emphasis on Others


    def lackigIdealOverboard(group_mean, personal_value): # group_mean: 1000명 평균, personal_value|:개인값
        ideal_mean = group_mean
        one_ps_char_desc = personal_value
        #최대, 최소값 기준으로 구간설정. 구간비율 30% => 0.3으로 설정
        min_ = int(ideal_mean-ideal_mean*0.5)
        # #print('min_', min_)
        max_ = int(ideal_mean+ideal_mean*0.5)
        # #print('max_: ', max_)
        div_ = int(((ideal_mean+ideal_mean*0.5)-(ideal_mean-ideal_mean*0.5))/3)
        # #print('div_:', div_)

        #결과 판단 Lacking, Ideal, Overboard
        cal_abs = abs(ideal_mean - one_ps_char_desc) # 개인 - 단체 값의 절대값계산

        # #print('cal_abs 절대값 :', cal_abs)
        compare7 = (one_ps_char_desc + ideal_mean)/6
        compare6 = (one_ps_char_desc + ideal_mean)/5
        compare5 = (one_ps_char_desc + ideal_mean)/4
        compare4 = (one_ps_char_desc + ideal_mean)/3
        compare3 = (one_ps_char_desc + ideal_mean)/2
        # #print('compare7 :', compare7)
        # #print('compare6 :', compare6)
        # #print('compare5 :', compare5)
        # #print('compare4 :', compare4)
        # #print('compare3 :', compare3)



        if one_ps_char_desc > ideal_mean: # 개인점수가 평균보다 클 경우는 overboard
            if cal_abs > compare3: # 37 개인점수가 개인평균차의 절대값보다 클 경우, 즉 차이가 많이 날경우
                # #print("Overboard: 2")
                result = 2 #overboard
                score = 1
            elif cal_abs > compare4: # 28
                # #print("Overvoard: 2")
                result = 2
                score = 2
            elif cal_abs > compare5: # 22
                # #print("Overvoard: 2")
                result = 2
                score = 3
            elif cal_abs > compare6: # 18
                # #print("Overvoard: 2")
                result = 2
                score = 4
            else:
                # #print("Ideal: 1")
                result = 1
                score = 5
        elif one_ps_char_desc < ideal_mean: # 개인점수가 평균보다 작을 경우 lacking
            if cal_abs > compare3: # 37 개인점수가 개인평균차의 절대값보다 클 경우, 즉 차이가 많이 날경우
                # #print("Lacking: 2")
                result = 0
                score = 1
            elif cal_abs > compare4: # 28
                # #print("Lacking: 2")
                result = 0
                score = 2
            elif cal_abs > compare5: # 22
                # #print("Lacking: 2")
                result = 0
                score = 3
            elif cal_abs > compare6: # 18
                # #print("Lacking: 2")
                result = 0
                score = 4
            else:
                # #print("Ideal: 1")
                result = 1
                score = 5
                
        else:
            # #print("Ideal: 1")
            result = 1
            score = 5

        return result, score


    #종합계산시작 lackigIdealOverboard(group_mean, personal_value)
    character_desc_result = lackigIdealOverboard(char_desc_ideal_mean, one_ps_char_desc)
    number_of_char_result = lackigIdealOverboard(num_of_char_ideal_mean, one_ps_num_of_char)
    emp_on_you_result = lackigIdealOverboard(emp_on_you_ideal_mean, one_ps_emp_on_you)
    emp_on_others_result = lackigIdealOverboard(emp_on_others_ideal_mean, one_ps_emp_on_others)

    fin_result = [character_desc_result, number_of_char_result, emp_on_you_result, emp_on_others_result]

    print('캐릭터 분석 결과', number_of_char_result)
    print('캐랙터 상대적 점수', fin_result[1][0])
    
    # each_fin_result의 결과는 순서대로 [Character Descriptiveness, Number of Characters, Emphasis on You, Emphasis on Others] 이다. 0: lacking, 1:ideal, 2:overbaord 
    each_fin_result = [fin_result[0][0], fin_result[1][0], fin_result[2][0], fin_result[3][0]]
    # 최종 character  전체 점수 계산
    overall_character_rating = [(fin_result[0][1]+ fin_result[1][1] + fin_result[2][1]+ fin_result[3][1])/4]

    result_final = each_fin_result + overall_character_rating + group_db_fin_result
    
    data = {
        # 이게 순서가 바뀌었음
        "number_of_chracters":one_ps_num_of_char, 
        "character_description":one_ps_char_desc,
        "emaphasis_on_you":one_ps_emp_on_you,
        "emaphasis_on_others":one_ps_emp_on_others,
        
        "result_number_of_chracters": fin_result[1][0],
        "result_character_description": result_final[1],
        "result_emaphasis_on_you" : result_final[2],
        "result_emaphasis_on_others" : result_final[3],
        
        "avg_character": result_final[4]
    }
    

    return data 


In [189]:
print("최종결과 : ", character_total_analysis(text))

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:236: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:243: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/gensim/models/base_any2vec.py:323: UserWarning: C extension not loaded, training will be slow. Install a C compil

중복제거된 캐릭터 표현 수: 7
전체 단어중에서 캐릭터 관련 단어가 얼마나 포함되었는지 비율: 6.69
레이블 추출: {('Bill', 'PERSON'), ('John', 'PERSON'), ('Tom', 'PERSON'), ('Eugene', 'WORK_OF_ART'), ('Peter', 'PERSON'), ('Jennifer', 'PERSON'), ('turkey', 'GPE'), ('Ted', 'PERSON'), ('Roy', 'PERSON'), ('Joe', 'PERSON'), ('Cyndi', 'GPE')}
에세이에 포함된 이름 수: 11
1명 에세이 총 캐릭터수: 7.53
전체 문장에서 캐릭터 단어 비율:::::: 0.84
emphasis on you 2
Emphasis on others 수집 단어들: ['cousin', 'cousin', 'her', 'they', 'they', 'they', 'they', 'their']
Emphasis on others: 8
한개의 에세이 캐릭터 수: 7.53
1000명 평균 캐릭터수_입력문장길이비율적용: 7
캐릭터 분석 결과 (1, 5)
캐랙터 상대적 점수 1
최종결과 :  {'number_of_chracters': 7.53, 'character_description': 36.0, 'emaphasis_on_you': 0.84, 'emaphasis_on_others': 3.35, 'result_number_of_chracters': 1, 'result_character_description': 1, 'result_emaphasis_on_you': 0, 'result_emaphasis_on_others': 0, 'avg_character': 2.5}
